# Texte et distance de jaccard

On va dans ce TD regarder les proximités de fables de la Fontaine en comparant les mots les composants.

Ceci est une prémière approche dans l'analyse de textes. Des méthodes bien plus *sioux* sont utilisées en vrai (mettre les mots au singulier, les verbe à l'infinitif, etc)

## création de la distance

Pour créer une distance entre fable on devra :

1. récupérer les fables
2. extraire les mots qui les composent
3. créer une distance

### Récupération des fables

Les fables sont disponibles là : https://www.gutenberg.org/files/56327/56327-h/56327-h.htm

On va récupérer le fichier html puis entraire le texte d celui-ci

#### page internet

En utilisant la bibliothèque [requests](https://requests-fr.readthedocs.io/en/latest/)

In [1]:
import requests

In [2]:
page = requests.get("https://www.gutenberg.org/files/56327/56327-h/56327-h.htm")

#### parser la page

Il faut récuperer les tokens html de la page qui compoent les fables.

En regardant la page dans un browser et les outils de développement on voit que :
* les titres des fables sont des des balises `<h3>`
* le texte des fables est dans le `<div>` juste en dessous (un `div` *frère*)

Il existe plusieurs bibliothèque pour parser du html. On va utiliser [beautifulsoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/).

**Références** :
* https://www.dataquest.io/blog/web-scraping-tutorial-python/
* http://sametmax.com/parser-du-html-avec-beautifulsoup/

In [3]:
from bs4 import BeautifulSoup

In [4]:
soup = BeautifulSoup(page.content, "html.parser")

In [5]:
[type(item) for item in list(soup.children)]

[bs4.element.Doctype,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString]

On cherche tous les fables. En regardant dans le html on remarque que les `<h3>` semblent correspondre aux titres des fables. On vérifie :

In [6]:
for x in soup.find_all('h3'):
    print(x)

<h3>LA CIGALE ET LA FOURMI.</h3>
<h3>LE CORBEAU ET LE RENARD.</h3>
<h3>LA GRENOUILLE QUI SE VEUT FAIRE AUSSI GROSSE QUE LE BŒUF.</h3>
<h3>LES DEUX MULETS.</h3>
<h3>LE LOUP ET LE CHIEN.</h3>
<h3>LA GÉNISSE, LA CHÈVRE ET LA BREBIS, EN SOCIÉTÉ AVEC LE LION.</h3>
<h3>LA BESACE.</h3>
<h3>L’HIRONDELLE ET LES PETITS OISEAUX.</h3>
<h3>LE RAT DE VILLE ET LE RAT DES CHAMPS.</h3>
<h3>LE LOUP ET L’AGNEAU.</h3>
<h3>L’HOMME ET SON IMAGE.</h3>
<h3>LE DRAGON A PLUSIEURS TÊTES, ET LE DRAGON A PLUSIEURS QUEUES.</h3>
<h3>LES VOLEURS ET L’ANE.</h3>
<h3>SIMONIDE PRÉSERVÉ PAR LES DIEUX.</h3>
<h3>LA MORT ET LE MALHEUREUX.</h3>
<h3>LA MORT ET LE BUCHERON.</h3>
<h3>L’HOMME ENTRE DEUX AGES ET SES DEUX MAITRESSES.</h3>
<h3>LE RENARD ET LA CIGOGNE.</h3>
<h3>L’ENFANT ET LE MAITRE D’ÉCOLE.</h3>
<h3>LE COQ ET LA PERLE.</h3>
<h3>LES FRELONS ET LES MOUCHES A MIEL.</h3>
<h3>LE CHÊNE ET LE ROSEAU.</h3>
<h3>CONTRE CEUX QUI ONT LE GOUT DIFFICILE.</h3>
<h3>CONSEIL TENU PAR LES RATS.</h3>
<h3>LE LOUP PLAIDANT CONTRE LE RENA

Le contenu est dans le `div` juste après (c'est un frère) :

In [7]:
for x in soup.find_all('h3'):
    print(x.text)
    print('----')
    poem = x.find_next_sibling()
    print(poem)
    print("=====")

LA CIGALE ET LA FOURMI.
----
<div class="poem marginleft210">
<div class="stanza">
<span class="i0">La cigale, ayant chanté</span><br/>
<span class="i4">Tout l’été,</span><br/>
<span class="i0">Se trouva fort dépourvue</span><br/>
<span class="i0">Quand la bise fut venue:</span><br/>
<span class="i0">Pas un seul petit morceau</span><br/>
<span class="i0">De mouche ou de vermisseau.</span><br/>
<span class="i0">Elle alla crier famine</span><br/>
<span class="i0">Chez la fourmi sa voisine,</span><br/>
<span class="i0">La priant de lui prêter</span><br/>
<span class="pagenum" id="Page_6">6</span>
<span class="i0">Quelque grain pour subsister</span><br/>
<span class="i0">Jusqu’à la saison nouvelle:</span><br/>
<span class="i0">Je vous paîrai, lui dit-elle,</span><br/>
<span class="i0">Avant l’oût, foi d’animal,</span><br/>
<span class="i0">Intérêt et principal.</span><br/>
<span class="i0">La fourmi n’est pas prêteuse;</span><br/>
<span class="i0">C’est là son moindre défaut:</span><br/>
<

On doit maintenant récupérer le poème. Il est dans une balise `span` de class différente à `pagenum`.

**Attention** : le dernier poème n'a pas de contenu. On doit traiter ce cas particulier pour qu'il n'y ait pas d'erreurs.

In [8]:
for x in soup.find_all('h3'):
    print(x.text)
    print('----')
    poem = x.find_next_sibling()
    contenu_div = poem.find('div')
    if contenu_div is None: # le dernier poeme n'a pas de contenu
        continue
    for y in contenu_div.find_all('span'):
        if 'pagenum' not in y.attrs['class']:
            print(y.text)
    print("====")

LA CIGALE ET LA FOURMI.
----
La cigale, ayant chanté
Tout l’été,
Se trouva fort dépourvue
Quand la bise fut venue:
Pas un seul petit morceau
De mouche ou de vermisseau.
Elle alla crier famine
Chez la fourmi sa voisine,
La priant de lui prêter
Quelque grain pour subsister
Jusqu’à la saison nouvelle:
Je vous paîrai, lui dit-elle,
Avant l’oût, foi d’animal,
Intérêt et principal.
La fourmi n’est pas prêteuse;
C’est là son moindre défaut:
Que faisiez-vous au temps chaud?
Dit-elle à cette emprunteuse.—
Nuit et jour à tout venant
Je chantois, ne vous déplaise.—
Vous chantiez! j’en suis fort aise.
Hé bien! dansez maintenant.
====
LE CORBEAU ET LE RENARD.
----
Maître corbeau, sur un arbre perché,
Tenoit en son bec un fromage.
Maître renard, par l’odeur alléché,
Lui tint à peu près ce langage:
Hé bonjour, monsieur du corbeau.
Que vous êtes joli! que vous me semblez beau!
Sans mentir, si votre ramage
Se rapporte à votre plumage,
Vous êtes le phénix des hôtes de ces bois.
A ces mots, le corbeau ne

On peut maintenant créer nos fables :

In [9]:
fables = {}

for x in soup.find_all('h3'):
    titre = x.text
    div_frere = x.find_next_sibling()
    contenu_div = div_frere.find('div')    
    if contenu_div is None: # le dernier poeme n'a pas de contenu
        continue
        
    contenu = ""
    for y in contenu_div.find_all('span'):
        if 'pagenum' not in y.attrs['class']:
            contenu += y.text + '\n'
    fables[titre] = contenu
    
fables

{'LA CIGALE ET LA FOURMI.': 'La cigale, ayant chanté\nTout l’été,\nSe trouva fort dépourvue\nQuand la bise fut venue:\nPas un seul petit morceau\nDe mouche ou de vermisseau.\nElle alla crier famine\nChez la fourmi sa voisine,\nLa priant de lui prêter\nQuelque grain pour subsister\nJusqu’à la saison nouvelle:\nJe vous paîrai, lui dit-elle,\nAvant l’oût, foi d’animal,\nIntérêt et principal.\nLa fourmi n’est pas prêteuse;\nC’est là son moindre défaut:\nQue faisiez-vous au temps chaud?\nDit-elle à cette emprunteuse.—\nNuit et jour à tout venant\nJe chantois, ne vous déplaise.—\nVous chantiez! j’en suis fort aise.\nHé bien! dansez maintenant.\n',
 'LE CORBEAU ET LE RENARD.': 'Maître corbeau, sur un arbre perché,\nTenoit en son bec un fromage.\nMaître renard, par l’odeur alléché,\nLui tint à peu près ce langage:\nHé bonjour, monsieur du corbeau.\nQue vous êtes joli! que vous me semblez beau!\nSans mentir, si votre ramage\nSe rapporte à votre plumage,\nVous êtes le phénix des hôtes de ces boi

## extraire les mots des fables

Pour cela on va comencer par mettre le texte en minuscule puis utiliser une expression régulière pour chercher tous les mots.

**Références** :
* https://fr.wikipedia.org/wiki/Expression_régulière
* https://www.lucaswillems.com/fr/articles/25/tutoriel-pour-maitriser-les-expressions-regulieres



In [10]:
import re

In [11]:
bag_of_word = {}
for titre, texte in fables.items():
    bag_of_word[titre] = set(re.findall(r'\b\w+\b',texte.lower()))

In [12]:
print(fables['LE RENARD ET LES POULETS D’INDE.'])

Contre les assauts d’un renard
Un arbre à des dindons servoit de citadelle.
Le perfide, ayant fait tout le tour du rempart,
Et vu chacun en sentinelle,
S’écria: Quoi! ces gens se moqueront de moi!
Eux seuls seront exempts de la commune loi!
Non, par tous les dieux! non. Il accomplit son dire.
La lune, alors luisant, sembloit, contre le sire,
Vouloir favoriser la dindonnière gent.
Lui, qui n’étoit novice au métier d’assiégeant,
Eut recours à son sac de ruses scélérates,
Feignit vouloir gravir, se guinda sur ses pattes,
Puis contrefit le mort, puis le ressuscité.
Arlequin n’eût exécuté
Tant de différents personnages.
Il élevoit sa queue, il la faisoit briller,
Et cent mille autres badinages;
Pendant quoi nul dindon n’eût osé sommeiller.
L’ennemi les lassoit en leur tenant la vue
Sur même objet toujours tendue.
Les pauvres gens étant à la longue éblouis,
Toujours il en tomboit quelqu’un: autant de pris,
Autant de mis à part: près de moitié succombe.
Le compagnon les porte en son garde-man

In [13]:
bag_of_word['LE RENARD ET LES POULETS D’INDE.']

{'accomplit',
 'alors',
 'arbre',
 'arlequin',
 'assauts',
 'assiégeant',
 'au',
 'autant',
 'autres',
 'ayant',
 'badinages',
 'briller',
 'cent',
 'ces',
 'chacun',
 'citadelle',
 'commune',
 'compagnon',
 'contre',
 'contrefit',
 'd',
 'de',
 'des',
 'dieux',
 'différents',
 'dindon',
 'dindonnière',
 'dindons',
 'dire',
 'du',
 'en',
 'ennemi',
 'et',
 'eut',
 'eux',
 'exempts',
 'exécuté',
 'eût',
 'faisoit',
 'fait',
 'favoriser',
 'feignit',
 'garde',
 'gens',
 'gent',
 'gravir',
 'guinda',
 'il',
 'l',
 'la',
 'lassoit',
 'le',
 'les',
 'leur',
 'loi',
 'longue',
 'lui',
 'luisant',
 'lune',
 'manger',
 'mille',
 'mis',
 'moi',
 'moitié',
 'moqueront',
 'mort',
 'métier',
 'même',
 'n',
 'non',
 'novice',
 'nul',
 'objet',
 'osé',
 'par',
 'part',
 'pattes',
 'pauvres',
 'pendant',
 'perfide',
 'personnages',
 'porte',
 'pris',
 'près',
 'puis',
 'quelqu',
 'queue',
 'qui',
 'quoi',
 'recours',
 'rempart',
 'renard',
 'ressuscité',
 'ruses',
 's',
 'sa',
 'sac',
 'scélérates',


On supprime les mots de longueur pus petit ou égal à 2 pour essayer de ne conserver que les *vrais* mots.

In [14]:
for titre in bag_of_word:
    new = set()
    for x in bag_of_word[titre]:
        if len(x) > 2:
            new.add(x)
    bag_of_word[titre] = new

### distance de jaccard

La  [distance de jaccard](https://fr.wikipedia.org/wiki/Indice_et_distance_de_Jaccard) est très utilisée pour comparer des objets décrit par des ensembles de propriétés. 

Elle vaut si :
* 1 : aucun mots en commun
* 0 : même vocabulaire

On commence par regarder les distance avec une fable de référence :

In [15]:
base = bag_of_word['LE RENARD ET LES POULETS D’INDE.']

similarite = []
for titre, texte in bag_of_word.items(): 
    similarite.append([1 - len(base.intersection(texte)) / len(base.union(texte)), titre])
    
similarite.sort()

In [16]:
similarite

[[0.0, 'LE RENARD ET LES POULETS D’INDE.'],
 [0.8899521531100478, 'LE RENARD ET LE BOUC.'],
 [0.8979591836734694, 'L’OURS ET LES DEUX COMPAGNONS.'],
 [0.9024390243902439, 'L’AIGLE ET LE HIBOU.'],
 [0.9027027027027027, 'LE MILAN ET LE ROSSIGNOL.'],
 [0.903061224489796, 'LE BERGER ET SON TROUPEAU.'],
 [0.9050279329608939, 'LE LION MALADE ET LE RENARD.'],
 [0.9078014184397163, 'LE GEAI PARÉ DES PLUMES DU PAON.'],
 [0.9080459770114943, 'LE LION S’EN ALLANT EN GUERRE.'],
 [0.9081081081081082, 'LA LIONNE ET L’OURSE.'],
 [0.9082125603864735, 'LE COCHON, LA CHÈVRE ET LE MOUTON.'],
 [0.9082278481012658, 'LES ANIMAUX MALADES DE LA PESTE.'],
 [0.908675799086758, 'LE LOUP ET LES BERGERS.'],
 [0.9090909090909091, 'LE CHAT ET LE RENARD.'],
 [0.9090909090909091, 'LE VIEILLARD ET SES ENFANTS.'],
 [0.9099099099099099, 'LES GRENOUILLES QUI DEMANDENT UN ROI.'],
 [0.91005291005291, 'LE RENARD AYANT LA QUEUE COUPÉE.'],
 [0.9104477611940298, 'LA GRENOUILLE ET LE RAT.'],
 [0.9111111111111111, 'LE FERMIER, LE

On crée la matrice de distance :

In [17]:
elements = list(bag_of_word.keys())

d = []

for x in elements:
    d.append([])
    base = bag_of_word[x]
    for y in elements:
        texte = bag_of_word[y]
        d[-1].append(1 - len(base.intersection(texte)) / len(base.union(texte)))
d

[[0.0,
  0.9236641221374046,
  0.900990099009901,
  0.9461538461538461,
  0.9521531100478469,
  0.9185185185185185,
  0.9174757281553398,
  0.919831223628692,
  0.987012987012987,
  0.9594594594594594,
  0.9281437125748503,
  0.9661016949152542,
  0.98989898989899,
  0.9,
  0.9307692307692308,
  0.9404761904761905,
  0.9125,
  1.0,
  0.9390243902439024,
  1.0,
  0.9281767955801105,
  0.9438202247191011,
  0.9156626506024097,
  0.935483870967742,
  0.9402985074626866,
  0.9085365853658537,
  0.9823008849557522,
  0.8974358974358975,
  0.9333333333333333,
  0.9456521739130435,
  0.9398907103825137,
  0.9397590361445783,
  0.9861111111111112,
  0.963855421686747,
  0.9021739130434783,
  0.9550561797752809,
  0.9534883720930233,
  0.9341317365269461,
  0.9193548387096774,
  0.9375,
  0.9176470588235294,
  0.9634146341463414,
  0.9585798816568047,
  0.9351351351351351,
  0.9314285714285714,
  0.9183673469387755,
  0.9722222222222222,
  0.9321266968325792,
  0.907563025210084,
  0.9622641509

## Analyse

**A vous** : 
* tentez une isomap en changeant le nombre de voisins à prendre en compte
* faites une hiérarchie.

### MDS

Faites une MDS en 2 dimensions de ces données. Conclusion ?

### isomap

* Faites une isomap en 2 dimensions de ces données en utilsant 2 plus proches voisins.
* faites un $k$-means à 4 classes et affichez les élements de la classe en-bas à droite
* pour cette classe, 

### hierarchie

* faite une hiérarchie et représentez la graphiquement 
* déterminez sa une hauteur de coupe de 0.875 est acceptable.

### partition d'une hiérarchie

Trouvez les classes à une hauteur d coupe de 0.875. Représentez la classe à 8 éléments. Quels sont les mots en communs ?